#### Быстрый старт: как использовать RAG-фреймворк в своём приложении

Этот пример демонстрирует базовое использование фреймворка с помощью YAML-конфигурации и контейнера внедрения зависимостей (DI).

---

Этот фреймворк предоставляет инфраструктуру для построения Retrieval-Augmented Generation (RAG) систем на Python. Он построен по принципам чистой архитектуры и использует YAML-конфигурации и DI-контейнеры для подключения компонентов (ретриверов, LLM, загрузчиков и т.д.).


#### Архитектура фреймворка

Фреймворк спроектирован в соответствии с принципами чистой архитектуры (Clean Architecture):

- **Порты (Ports)** — это интерфейсы, описывающие, что именно должен уметь компонент (например, LLMPort, RetrieverPort).
- **Адаптеры (Adapters)** — это конкретные реализации этих интерфейсов, например `HuggingFaceInferenceAdapter`.
- **DIContainer** позволяет собирать адаптеры по конфигурации, не нарушая изоляцию и масштабируемость.

Таким образом, вы можете легко менять адаптеры (например, вместо HuggingFace использовать OpenAI или свою реализацию), не затрагивая бизнес-логику.

#### Глоссарий ключевых компонентов фреймворка

- **IngestionService** — отвечает за подготовку данных: загрузку, разбиение на чанки и индексацию в ретривере.
- **AnswerService** — отвечает за основной пайплайн ответа: извлечение релевантных документов и генерация ответа через LLM.
- **Retriever** — компонент, отвечающий за поиск по базе документов. Интерфейс описан в `RetrieverPort`.
- **LLM** — large language model. Интерфейс генерации ответа описан в `LLMPort`.
- **DIContainer** — dependency injection-контейнер, собирающий зависимости и управляющий жизненным циклом компонентов.
- **ADAPTER_REGISTRY** — глобальный реестр реализаций интерфейсов (адаптеров), доступных для подключения через конфигурацию.

Проще всего начать использование фреймворка с YAML-конфигурации и контейнера зависимостей.

Все, что от вас требуется:
- загрузить настроенный файл конфигурации,
- создать экземпляр конструктора зависимостей `DIContainer` и вызвать метод построения этих зависимостей на основе файла конфигурации `build_app_service`,
- воспользоваться методом генерации ответа на основе найденных релевантных документов `generate_answer`.

In [1]:
from ragbee_fw import load_config

In [2]:
app_config_path = "/workspace/src/ragbee_fw/config/app_config.yml"
app_config = load_config(app_config_path)
app_config.llm.token = "your_token"

In [3]:
from ragbee_fw import DIContainer


container = DIContainer(app_config)
app = container.build_app_service()

В результате мы получаем экземпляр класса `AnswerService`, который создан для основного рабочего процесса:
- поиск релевантных пользовательскому вопросу участков текста для генерации ответа LLM,
- генерация ответа с использованием LLM.

In [14]:
type(app)

ragbee_fw.core.services.answer_service.AnswerService

Теперь мы можем просто передать вопрос в метод `generate_answer` созданного экземпляра класса `AnswerService` и получить ответ от LLM:

In [ ]:
response = app.generate_answer(query="какие были Испанские завоевания в Америке?")

In [11]:
response

'Испанские завоевания в Америке включали в себя несколько важных событий и кампаний, которые привели к колонизации значительной части американского континента. Вот ключевые моменты:\n\n1. **Экспедиция Кортеса в Мексику (1518 г.)**: Испанский губернатор Кубы отправил экспедицию под руководством Фернандо Кортеса в Мексику. Кортес высадился с 600 человек и, чтобы исключить возможность отступления, сжег все корабли.\n\n2. **Завоевание государства ацтеков (1521 г.)**: Кортес двинулся вглубь страны и достиг столицы ацтеков, Мехико. Первоначально ацтеки приняли испанцев за богов, но мирные отношения быстро закончились, и испанцы учинили погром. В 1521 году Кортес окончательно захватил и разграбил Мехико, а страну объявили собственностью испанского короля.\n\n3. **Завоевание империи инков (1532–1534 гг.)**: Франциско Писарро вторгся в империю инков в Южной Америке, захватил в плен их правителя, и к 1534 году государство инков было уничтожено.\n\n4. **Завоевание Венесуэлы (1525–1535 гг.)**: Исп

---
*Обратите внимание, что DI контейнер хранит внутри себя все созданные объекты в словаре `_cache`. Это свойство может быть полезным при создании пользовательских модулей и интеграции пользовательских зависимостей:*

In [24]:
contain_obj = container._cache
display(contain_obj.keys())

dict_keys(['data_loader', 'text_chunker', 'retriever', 'retriever_with_index', 'llm'])

In [25]:
retriever = contain_obj.get("retriever")
display(retriever)

#### Низкоуровневый контроль создания зависимостей и кастомная реальзация модулей

Вы можете отказаться от использования DI-контейнера и создать зависимости вручную. Это обеспечит максимальный контроль и гибкость при интеграции собственных модулей.

Однако, это потребует дополнительного изучения структуры фрейворка и его реализации, включая погружение в строгие но гибкие отношения портов и адаптеров в духе гексогональной архитектуры.

In [4]:
from ragbee_fw.infrastructure.data_loader.file_loader import FileSystemLoader
from ragbee_fw.infrastructure.text_splitter.recursive_text_splitter import RecursiveTextSplitter
from ragbee_fw.infrastructure.retriever.bm25_client import BM25Client 
from ragbee_fw.infrastructure.llm_clients.huggingface_client import HuggingFaceInferenceAdapter

from ragbee_fw import IngestionService
from ragbee_fw import AnswerService

In [ ]:
loader = FileSystemLoader()
chanker = RecursiveTextSplitter()
retriever = BM25Client()
llm = HuggingFaceInferenceAdapter(model_name="meta-llama/Llama-4-Scout-17B-16E-Instruct",
                                    provider="cerebras",
                                    token="your_token",)


На данном этапе, вы можете создать и внедрить любой модуль в пайплайн, который так или иначе вписывается в задачи:
- загрузки, рабиения и индексации данных - для сервиса `IngestionService`, 
- поиска по базе/индексу и генерация ответа - для сервиса `AnswerService`.  
  
Все, что вам необходимо - следовать установленным правилам портами поключения к пайплайну, описанным в `ragbee_fw.core.ports`.

In [ ]:
ingestion = IngestionService(
    loader=loader,
    chunker=chanker,
    retriever=retriever
)
retriever = ingestion.build_index("/workspace/documents")

In [ ]:
responsible = AnswerService(
    retriever=retriever,
    llm=llm
)
response = responsible.generate_answer(
    query="какие были Испанские завоевания в Америке?"
)

In [ ]:
response

'Испанские завоевания в Америке включали в себя несколько ключевых событий и завоеваний, которые существенно повлияли на историю региона. Вот основные моменты:\n\n1. **Экспедиция Кортеса в Мексику (1518 г.)**: Испанский губернатор острова Кубы отправил экспедицию под предводительством Фернандо Кортеса в Мексику. Кортес сжег все корабли после высадки, чтобы исключить возможность возвращения, и двинулся вглубь страны к государству ацтеков.\n\n2. **Захват Мехико (1521 г.)**: Испанцы беспрепятственно вошли в столицу ацтеков, Мехико, где их приняли за богов. Однако мирные отношения быстро закончились, и испанцы учинили погром. В 1521 году Кортес окончательно захватил и разграбил Мехико, объявив страну собственностью испанского короля.\n\n3. **Завоевание империи инков (1532 г.)**: Франциско Писарро вторгся в империю инков в Южной Америке и захватил в плен их правителя. К 1534 году государство инков было уничтожено.\n\n4. **Завоевание Венесуэлы (1525–1535 гг.)**: Испанцы окончательно завоевал

#### Создание пользовательского модуля и регистрация в контейнере зависимостей

Самый простой способ добавить свою реализацию модуля — зарегистрировать её в DI-контейнере через ADAPTER_REGISTRY. Это снижает порог входа и упрощает интеграцию.

Все что вам нужно, это:
- написать собственный класс модуля. Следите, что вы соответствуете предопределенным соглашениям в разделе порты `ragbee_fw.core.ports`. Так же, в этом разделе вы можете найти абстрактные базовые классы, использование которых может дать больше ясности и прозрачности для следования соглашениям протоколов портов.
- зарегистрировать свою реализацию модуля в `ADAPTER_REGISTRY`.
- составить файл конфигурации приложения `config.yml` с учетом кастомного модуля.
- воспользоваться стандартной процедурой генерации ответов с DI контейнером.

In [5]:
from ragbee_fw.core.ports.llm_port import LLMPort, BaseLLM
from ragbee_fw import DIContainer, ADAPTER_REGISTRY


In [2]:
display(ADAPTER_REGISTRY)

{'data_loader': {'file_loader': ragbee_fw.infrastructure.data_loader.file_loader.FileSystemLoader},
 'text_chunker': {'recursive_splitter': ragbee_fw.infrastructure.text_splitter.recursive_text_splitter.RecursiveTextSplitter},
 'retriever': {'bm25': ragbee_fw.infrastructure.retriever.bm25_client.BM25Client},
 'llm': {'HF': ragbee_fw.infrastructure.llm_clients.huggingface_client.HuggingFaceInferenceAdapter}}

In [ ]:
class MyDummyResponce(LLMPort):
    def generate(self, prompt: str):
        # custom logic goes here
        return f"I got {len(prompt)} chars in out prompt: {prompt}"


In [4]:
DIContainer.register_adapter(component="llm", 
                           adapter_type="dummy",
                           cls=MyDummyResponce)

In [5]:
display(ADAPTER_REGISTRY)

{'data_loader': {'file_loader': ragbee_fw.infrastructure.data_loader.file_loader.FileSystemLoader},
 'text_chunker': {'recursive_splitter': ragbee_fw.infrastructure.text_splitter.recursive_text_splitter.RecursiveTextSplitter},
 'retriever': {'bm25': ragbee_fw.infrastructure.retriever.bm25_client.BM25Client},
 'llm': {'HF': ragbee_fw.infrastructure.llm_clients.huggingface_client.HuggingFaceInferenceAdapter,
  'dummy': __main__.MyDummyResponce}}

Теперь ваш модуль зарегистрирован в реестре и может быть добавлен к пайплайну на замену предопределенному в фреймворке.

Следующим шагом будет описать пайплайн в config.yml с учетом вашего пользовательского модуля. 
```yaml
# config_app.yml
llm:
  type: dummy
  model_name: any_name
  token: ""
  provider: ""
  prompt: ""
  max_new_tokens: 0
  ```

В этом примере мы напрямую изменим уже загруженный Pydantic-объект конфигурации, не редактируя YAML-файл.

In [6]:
from ragbee_fw import load_config

app_config_path = "/workspace/src/ragbee_fw/config/app_config.yml"
app_config = load_config(app_config_path)

In [7]:
from ragbee_fw.core.models.app_config import LLM


app_config.llm = LLM(**{
    "type": "dummy",
    "model_name": "any_name",
    "token": "",
    "provider": "",
    "prompt": "",
    "max_new_tokens": 0,
})

> 💡 Обратите внимание: `load_config(...)` возвращает объект Pydantic-модели. Это значит, что вы можете модифицировать его в коде как обычный Python-объект:
> - вручную заменять поля (как `app_config.llm = ...`),
> - валидировать значения,
> - использовать `.dict()` и `.json()` для экспорта обратно.

Если вы захотите сохранить конфигурацию обратно в YAML, используйте:

```python
from pathlib import Path
import yaml

with Path("new_config.yml").open("w", encoding="utf-8") as f:
    yaml.safe_dump(app_config.dict(), f, allow_unicode=True)
```

In [8]:
display(app_config.llm)

LLM(type='dummy', model_name='any_name', token='', provider='', base_url=None, prompt='', max_new_tokens=0, return_full_response=False, params=None)

Теперь у нас есть класс конфигурации параметров приложения с учетом кастомного модуля генерации ответов на основе логики (шаблона), определенной в `MyDummyResponce`. Дальнейшие действия аналогичны базовому использованию API фреймворка:

In [9]:
container = DIContainer(app_config)
app = container.build_app_service()

In [10]:
response = app.generate_answer(query="какие были Испанские завоевания в Америке?")

In [11]:
response

'I got 4150 chars in out prompt: Based on the following fragments:\n\n[1]  прибыль.   Испанские завоевания в Америке.\n   В 1518 г. испанский губернатор острова Кубы послал экспедицию из 600 человек во главе с Фернандо Кортесом в только что открытую Мексику. Кортес сжег после высадки все корабли, чтобы нельзя было вернуться домой, и двинулся в глубь страны к государству ацтеков. Испанцы беспрепятственно проникли в столицу ацтеков Мехико. Ацтеки приняли белокожих испанцев за богов. Но мирные отношения быстро закончились. Испанцы учинили страшный погром. В 1521 г. Кортес окончательно захватил и разграбил Мехико. Страна была объявлена собственностью испанского короля.\n   В 1532 г. Франциска Писарро вторгся в империю инков в Южной Америке и захватил в плен их правителя. К 1534 г. государство инков было уничтожено. В 1525–1535 гг. испанцы окончательно завоевали Венесуэлу. Здесь было найдено золото и возникла легенда о волшебной стране Эльдорадо, в которой много золота. Поиски Эльдорадо был